### Create True Graph

dataset - http://www.geniaproject.org/genia-corpus/relation-corpus

In [1]:
# !pip install networkx --user

In [2]:
import os
from xml.dom import minidom
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
# file_locs = ['/home/grace/Documents/DAEB/jupyter/dataset/BioNLP-ST_2011_Entity_Relations_training_data']
file_locs = ['/Users/grace/workspace/bio_dataset/GENIA_relation_annotation_training_data/BioNLP-ST_2011_Entity_Relations_training_data',
            '/Users/grace/workspace/bio_dataset/GENIA_relation_annotation_test_data/BioNLP-ST_2011_Entity_Relations_test_data/']
total_proteins = []
total_entities = []
sents = []
for file_loc in file_locs:
    medlines = os.listdir(file_loc)
    print(len(medlines))
#     medlines = list(filter(lambda x: x.endswith('.txt'), medlines))
    print(len(medlines))

    for med in medlines:
        f = open(file_loc+'/'+med, 'r')
        if med.endswith('.txt'):
            lines = ' '.join(f.readlines()).replace('\n', ' ')
            sents.append({'id':med.replace('.txt', ''),
                          'sent':lines.lower()})
            
        elif med.endswith('.a1'):
            entities = {}
            for line in f.readlines():
                items = line.split('\t')
#                 print(items)
#                 entity_type = items[1]
#                 entity_value = items[4]
                entities[items[2].strip().lower()] = items[1].split(' ')[0].lower()
    
            total_proteins.append({'id': med.replace('.a1', ''),
                             'protein_val': ' *,* '.join(entities.keys()),
                             'protein_type': ' *,* '.join(entities.values())})
            
        elif med.endswith('.rel'):
            entities = {}
            for line in f.readlines():
                if line.startswith('T'):
                    items = line.split('\t')
#                     print(items)
    #                 entity_type = items[1]
    #                 entity_value = items[4]
                    entities[items[2].strip().lower()] = items[1].split(' ')[0].lower()
    
            total_entities.append({'id': med.replace('.rel', ''),
                             'entity_val': ' *,* '.join(entities.keys()),
                             'entity_type': ' *,* '.join(entities.values())})
        f.close()

2402
2402
522
522


In [4]:
df_sent = pd.DataFrame(sents)
print(df_sent.shape)
df_sent.head()
# df_sent.sent.isnull()

(1060, 2)


,id,sent
0,PMID-8709209,"various modes of basic helix-loop-helix protein-mediated regulation of murine leukemia virus transcription in lymphoid cell lines. the transcriptionally regulatory regions of the lymphomagenic akv and sl3-3 murine leukemia retroviruses (mlvs) contain two types of e-box consensus motifs, cagatg. one type, ea/s, is located in the upstream promoter region, and the other, e(gre), is located in a tandem repeat with enhancer properties. we have examined the requirements of the individual e-boxes in mlv transcriptional regulation. in lymphoid cell lines only, the e(gre)-binding protein complexes included alf1 or heb and e2a basic helix-loop-helix proteins. ectopic alf1 and e2a proteins required intact e(gre) motifs for mediating transcriptional activation. alf1 transactivated transcription of akv mlv through the two e(gre) motifs equally, whereas e2a protein required the promoter-proximal e(gre) motif. in t- and b-cell lines, the e(gre) motifs were of major importance for akv mlv transcriptional activity, while the ea/s motif had some effect. in contrast, neither e(gre) nor ea/s motifs contributed pronouncedly to akv mlv transcription in nih 3t3 cells lacking dna-binding alf1 or heb and e2a proteins. the id1 protein was found to repress alf1 activity in vitro and in vivo. moreover, ectopic id1 repressed e(gre)-directed but not ea/s-directed mlv transcription in lymphoid cell lines. in conclusion, e(gre) motifs and interacting basic helix-loop-helix proteins are important determinants for mlv transcriptional activity in lymphocytic cell lines."
1,PMID-1946356,"negative regulation of human immunodeficiency virus type 1 expression in monocytes: role of the 65-kda plus 50-kda nf-kappa b dimer. although monocytic cells can provide a reservoir for viral production in vivo, their regulation of human immunodeficiency virus type 1 (hiv-1) transcription can be either latent, restricted, or productive. these differences in gene expression have not been molecularly defined. in thp-1 cells with restricted hiv expression, there is an absence of dna-protein binding complex formation with the hiv-1 promoter-enhancer associated with markedly less viral rna production. this absence of binding was localized to the nf-kappa b region of the hiv-1 enhancer; the 65-kda plus 50-kda nf-kappa b heterodimer was preferentially lost. adding purified nf-kappa b protein to nuclear extracts from cells with restricted expression overcomes this lack of binding. in addition, treatment of these nuclear extracts with sodium deoxycholate restored their ability to form the heterodimer, suggesting the presence of an inhibitor of nf-kappa b activity. furthermore, treatment of nuclear extracts from these cells that had restricted expression with lipopolysaccharide increased viral production and nf-kappa b activity. antiserum specific for nf-kappa b binding proteins, but not c-rel-specific antiserum, disrupted heterodimer complex formation. thus, both nf-kappa b-binding complexes are needed for optimal viral transcription. binding of the 65-kda plus 50-kda heterodimer to the hiv-1 enhancer can be negatively regulated in monocytes, providing one mechanism restricting hiv-1 gene expression."
2,PMID-7543515,"costimulation of human cd4+ t cells with lfa-3 and b7 induce distinct effects on ap-1 and nf-kappa b transcription factors. we have earlier shown that stimulation of human cd4+ t cells with sea presented on chinese hamster ovary (cho)-dr transfectants coexpressing either b7 or lfa-3 resulted in distinct cytokine profiles. we now demonstrate that b7, but not lfa-3, strongly costimulated il-2 transcription and mrna expression in cd4+ t cells. maximal increase in il-2 transcription was recorded with cho-dr/b7/lfa-3, suggesting a cooperative effect of b7 and lfa-3 at the transcriptional level. gel-shift analysis demonstrated that stimulation of cd4+ t cells with cho-dr and staphylococcal enterotoxin a was sufficient to induce significant amounts of nf-ka

In [5]:
df_proteins = pd.DataFrame(total_proteins)
df_proteins.head()

,id,protein_type,protein_val
0,PMID-8943338,"protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein","elf-1 *,* stat5 *,* interleukin-2 receptor alpha *,* interleukin 2 receptor alpha-chain *,* il-2r alpha *,* interleukin 2 *,* il-2 *,* hmg-i(y) *,* cat"
1,PMID-9224203,,
2,PMID-1653056,"protein *,* protein","tumor necrosis factor alpha *,* tnf alpha"
3,PMID-9872676,"protein *,* protein *,* protein *,* protein *,* protein *,* protein","interleukin-10 *,* inhibitory kappab-alpha *,* il-10 *,* tumor necrosis factor-alpha *,* tnf-alpha *,* ikappab-alpha"
4,PMID-8264604,"protein *,* protein *,* protein *,* protein","pu.1 *,* macrophage colony-stimulating factor receptor *,* macrophage colony-stimulating factor (m-csf) receptor *,* m-csf receptor"


In [6]:
df_entities = pd.DataFrame(total_entities)
df_entities.head()

,entity_type,entity_val,id
0,"entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity","enhancer *,* promoter *,* promoters *,* enhancers *,* its own promoter. *,* immunoglobulin kappa enhancer *,* immunoglobulin heavy chain enhancer *,* kappa gene promoter *,* genes",PMID-1668145
1,"entity *,* entity *,* entity *,* entity","nf-kappa b *,* transcription factor *,* cytokine promoters *,* nf-kappa b-controlled cytokine promoters",PMID-9247567
2,"entity *,* entity *,* entity *,* entity *,* entity *,* entity","human immunodeficiency virus type 1 enhancer *,* site *,* kappa b sites *,* region *,* cd3 *,* cis-acting negative regulatory element",PMID-1533884
3,"entity *,* entity *,* entity","cytokine gene *,* cytokine genes *,* proinflammatory cytokine gene",PMID-8038234
4,"entity *,* entity *,* entity *,* entity","transcriptional complex *,* promoter *,* regulatory element *,* regulatory elements",PMID-7706710


## True Graph with Protein Entities Only

In [7]:
pro_entities = []
def reorganize_by_entity(row):
    val_list = list(filter(lambda x: x.strip(), row['protein_val'].split(' *,* ')))
    pro_entities.extend(list(map(lambda x:{'id':row['id'], 'val': x}, val_list)))

df_proteins.apply(reorganize_by_entity, axis=1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
        ... 
1050    None
1051    None
1052    None
1053    None
1054    None
1055    None
1056    None
1057    None
1058    None
1059    None
Length: 1060, dtype: object

In [8]:
df_pro_entities = pd.DataFrame(pro_entities)
df_pro_entities.head()

,id,val
0,PMID-8943338,elf-1
1,PMID-8943338,stat5
2,PMID-8943338,interleukin-2 receptor alpha
3,PMID-8943338,interleukin 2 receptor alpha-chain
4,PMID-8943338,il-2r alpha


In [9]:
df_pro_entities.loc[df_pro_entities.val=='']

,id,val


In [10]:
#ids = 800
len(df_pro_entities.id.unique())

1016

In [11]:
#values = 1540
len(df_pro_entities.val.unique())

1824

In [12]:
df_pro_entities.shape

(4753, 2)

In [13]:
df_sorted = df_pro_entities.groupby(df_pro_entities.val).size().reset_index(name='count_').sort_values(['count_'], ascending=False)
df_sorted.head()

,val,count_
850,il-2,149
1314,p50,97
1329,p65,94
1693,tnf-alpha,76
872,il-4,60


In [14]:
df_sorted.shape

(1824, 2)

In [15]:
once_appeared_values = df_sorted[df_sorted.count_==1].val.unique()
len(df_pro_entities.loc[df_pro_entities.val.isin(once_appeared_values)].id.unique())

592

In [16]:
copy_pro_entities = df_pro_entities.copy(deep=True)

In [17]:
copy_pro_entities.head()

,id,val
0,PMID-8943338,elf-1
1,PMID-8943338,stat5
2,PMID-8943338,interleukin-2 receptor alpha
3,PMID-8943338,interleukin 2 receptor alpha-chain
4,PMID-8943338,il-2r alpha


In [18]:
ordered_vals = list(df_sorted.val.unique())
# ordered_vals

In [19]:
def find_most_effective_recursively(copy_df, i):
#     print(ordered_vals[i])
    found = copy_df.loc[copy_df.val==ordered_vals[i]]
#     print(found)
    if len(found.id.unique()) > 1:
        return find_most_effective_recursively(copy_df.loc[copy_df.id.isin(found.id.unique())], i+1)
    elif len(found.id.unique()) ==1:
        return found.iloc[0]
    else:
#         print(type(copy_df.groupby(copy_df.id).count().reset_index().sort_values(['val'], ascending=False).loc[0]))
        return copy_df.groupby(copy_df.id).count().reset_index().sort_values(['val'], ascending=False).loc[0]
    
print(find_most_effective_recursively(copy_pro_entities, 3))

id     PMID-8038234
val    12          
Name: 0, dtype: object


In [20]:
already_annotated = []
iter_no = 0
while(len(ordered_vals)>1):
    val = ordered_vals[0]

    found_df = find_most_effective_recursively(copy_pro_entities, 0)
    found_id = found_df.id
    annotated_keywords = copy_pro_entities.loc[copy_pro_entities.id.eq(found_id)].val.unique()
    if len(annotated_keywords)==0:
        annotated_keywords.remove(val)
        continue
        
    already_annotated.append({'id': found_id, 
                              'keywords': annotated_keywords})

    copy_pro_entities = copy_pro_entities.loc[(copy_pro_entities.id!=found_id) & (~copy_pro_entities.val.isin(annotated_keywords))]
    copy_pro_entities.sort_index(inplace=True)
    
    for an in annotated_keywords:
        ordered_vals.remove(an)
#     print(len(ordered_vals))
    iter_no += 1
    
print(iter_no)

677


In [21]:
677/1060

0.6386792452830189

In [22]:
(677-592)

85

## PageRank

In [23]:
import networkx as nx

In [24]:
G = nx.DiGraph()

In [25]:
for k in df_pro_entities.id.unique():
    G.add_node(k)

In [26]:
for val, grp in df_pro_entities.groupby('val'):
#     print(grp)
    if len(grp) > 1:
#         print(grp)
        id_list = grp.id.unique()
        for i in range(len(id_list)-1):
            G.add_edge(id_list[i], id_list[i+1])
            G.add_edge(id_list[i+1], id_list[i])

In [27]:
import operator

pageranked_list = nx.pagerank(G)
max(pageranked_list.items(), key=operator.itemgetter(1))

('PMID-7721885', 0.004118988562885963)

In [28]:
min(pageranked_list.items(), key=operator.itemgetter(1))

('PMID-8195215', 0.00015612802498048405)

In [29]:
# sorted_pagedranked = sorted(pageranked_list, key=operator.itemgetter(1))
sorted_pagedranked = sorted(pageranked_list.items(), key=operator.itemgetter(1), reverse=True)
sorted_pagedranked[:10]

[('PMID-7721885', 0.004118988562885963),
 ('PMID-10233888', 0.003392423167119465),
 ('PMID-10097788', 0.003348454099375892),
 ('PMID-8513868', 0.0030160294180792066),
 ('PMID-10339475', 0.0028973685271362582),
 ('PMID-8038234', 0.002888896679855576),
 ('PMID-8083467', 0.0028456806977114426),
 ('PMID-7517211', 0.0028009352354447397),
 ('PMID-9724088', 0.0026069928206084664),
 ('PMID-8871617', 0.002594185168450978)]

In [30]:
pageranked_list['PMID-7721885']

0.004118988562885963

## PageRanked order -> Count the number of iteration

In [31]:
copy_pro_entities = df_pro_entities.copy(deep=True)
ordered_vals = list(df_sorted.val.unique())

already_annotated = []
iter_no = 0
i = 0
while(len(ordered_vals)>0):
    found_id = sorted_pagedranked[i][0]

    annotated_keywords = copy_pro_entities.loc[copy_pro_entities.id.eq(found_id)].val.unique()
    if len(annotated_keywords)==0:
        i += 1
        continue
        
    already_annotated.append({'id': found_id, 
                              'keywords': annotated_keywords})

    copy_pro_entities = copy_pro_entities.loc[(copy_pro_entities.id!=found_id) & (~copy_pro_entities.val.isin(annotated_keywords))]
    copy_pro_entities.sort_index(inplace=True)
    
    for an in annotated_keywords:
        ordered_vals.remove(an)
#     print(len(ordered_vals))
    iter_no += 1
print(iter_no)

672
